In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Load Dataset
disease_df = pd.read_csv('disease_diagnosis.csv')

In [3]:
# Preprocessing

# Split Blood Pressure into Systolic and Diastolic
bp_split = disease_df['Blood_Pressure_mmHg'].str.split('/', expand=True).astype(float)
disease_df['BP_Systolic'] = bp_split[0]
disease_df['BP_Diastolic'] = bp_split[1]

# One-Hot Encode Symptoms
all_symptoms = pd.unique(
    disease_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].values.ravel()
)

for sym in all_symptoms:
    disease_df[f"symptom_{sym}"] = disease_df[['Symptom_1', 'Symptom_2', 'Symptom_3']].isin([sym]).any(axis=1).astype(int)

# Feature and Target Selection
features = ['Heart_Rate_bpm', 'Body_Temperature_C',
            'Oxygen_Saturation_%', 'BP_Systolic', 'BP_Diastolic'] + \
           [f"symptom_{sym}" for sym in all_symptoms]

X = disease_df[features]
# Target Columns
y = disease_df[['Diagnosis', 'Severity']]

# Encode each target
diagnosis_encoder = LabelEncoder()
severity_encoder = LabelEncoder()

y['Diagnosis'] = diagnosis_encoder.fit_transform(y['Diagnosis'])
y['Severity'] = severity_encoder.fit_transform(y['Severity'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17032\1684337388.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Diagnosis'] = diagnosis_encoder.fit_transform(y['Diagnosis'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_17032\1684337388.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Severity'] = severity_encoder.fit_transform(y['Severity'])


In [4]:
# Initialize Naive Bayes Classifier
nb = GaussianNB()
multi_nb = MultiOutputClassifier(nb)

#Train Model
multi_nb.fit(X_train,y_train)

# Predictions
y_pred = multi_nb.predict(X_test)

# Decode Predictions
y_pred_diagnosis = diagnosis_encoder.inverse_transform(y_pred[:, 0])
y_pred_severity = severity_encoder.inverse_transform(y_pred[:, 1])

In [5]:
# Diagnosis Evaluation
print("\nDiagnosis Classification Report:")
print(classification_report(y_test['Diagnosis'], y_pred[:, 0]))

print("\nDiagnosis Confusion Matrix:")
print(confusion_matrix(y_test['Diagnosis'], y_pred[:, 0]))

# Severity Evaluation
print("\nSeverity Classification Report:")
print(classification_report(y_test['Severity'], y_pred[:, 1]))

print("\nSeverity Confusion Matrix:")
print(confusion_matrix(y_test['Severity'], y_pred[:, 1]))


Diagnosis Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        64
           1       0.97      0.71      0.82       238
           2       0.60      0.97      0.74        58
           3       0.50      0.45      0.48        11
           4       1.00      0.72      0.84        29

    accuracy                           0.79       400
   macro avg       0.74      0.77      0.73       400
weighted avg       0.85      0.79      0.79       400


Diagnosis Confusion Matrix:
[[ 64   0   0   0   0]
 [ 32 168  33   5   0]
 [  2   0  56   0   0]
 [  1   0   5   5   0]
 [  3   5   0   0  21]]

Severity Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.86      0.92       267
           1       0.52      1.00      0.69        58
           2       1.00      0.80      0.89        75

    accuracy                           0.87       400
   macro avg       0.84    